In [18]:
import pandas as pd

# ✅ Load CSV (using forward slashes so no escape sequence issues)

# Load datasets
matches = pd.read_csv("IPL Dataset/matches.csv")
deliveries = pd.read_csv("IPL Dataset/deliveries.csv")

# ✅ Make sure all columns show in one line (no wrapping with \)
pd.set_option("display.max_columns", 600)
pd.set_option("display.width", 600)

matches.head(1)  # Display the first few rows of the DataFrame


,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN


Player won MOM without winning team


In [21]:

# Step 1: Take match_id, winner, player_of_match
mom_df = matches[['id','Season','winner','player_of_match']].rename(columns={'Season':'Season'})

# Step 2: From deliveries, create a mapping of player -> team (per match)
# For batsmen
batsman_team = deliveries.groupby(['match_id','batsman'])['batting_team'].first().reset_index()
batsman_team.rename(columns={'batsman':'player','batting_team':'team'}, inplace=True)

# For bowlers
bowler_team = deliveries.groupby(['match_id','bowler'])['bowling_team'].first().reset_index()
bowler_team.rename(columns={'bowler':'player','bowling_team':'team'}, inplace=True)

# Combine both (some PoM are bowlers, some batsmen)
player_team_map = pd.concat([batsman_team, bowler_team]).drop_duplicates()

# Step 3: Merge PoM info with their team
mom_with_team = mom_df.merge(player_team_map, left_on=['id','player_of_match'], right_on=['match_id','player'], how='left')

# Step 4: Filter where player's team != winner (MoM from losing side)
losing_side_mom = mom_with_team[mom_with_team['team'] != mom_with_team['winner']]

# Step 5: Show results with Season (capitalized)
print(losing_side_mom[['Season','player_of_match','team','winner']].tail(5))


       Season player_of_match                     team               winner
598  IPL-2016       CH Morris         Delhi Daredevils        Gujarat Lions
615  IPL-2016         A Zampa  Rising Pune Supergiants  Sunrisers Hyderabad
621  IPL-2016         HM Amla          Kings XI Punjab  Sunrisers Hyderabad
660  IPL-2018      AS Rajpoot          Kings XI Punjab  Sunrisers Hyderabad
744  IPL-2019             NaN                      NaN                  NaN
